In [0]:
#!tar -xf cifar-10-python.tar.gz

# Inception V1 ( With dimension reductions )
Adding 1x1 conv stage 

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )
        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )
        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )
        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(160000, 10)
        

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        out = torch.cat([y1,y2,y3,y4], 1)
        #print(out.size())
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [0]:
# Was testing the model sizes
def test():
    net = Inception(3,  64,  96, 128, 16, 32, 32)
    x = torch.randn(1,3,32,32)
    y = net(x)
    print(y.size())
    print(y)
test()

torch.Size([1, 256, 32, 32])
torch.Size([1, 10])
tensor([[ 0.2319, -0.3461, -0.3062, -0.1209,  0.1695, -0.0083,  0.1265, -0.0462,
         -0.0766,  0.0283]], grad_fn=<AddmmBackward>)


In [0]:
x = torch.randn(1,3,32,32)

# GoogleNet (Inception V1)

In [0]:
'''GoogLeNet with PyTorch.'''
import torch
import torch.nn as nn
import torch.nn.functional as F
class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )
        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )
        #print(n5x5)
        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5, kernel_size=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )
        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
        
    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        '''print(y1.size())
        print(y2.size())
        print(y3.size())
        print(y4.size())'''
        return torch.cat([y1,y2,y3,y4], 1)
class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )
        self.a3 = Inception(192,  64,  96, 128, 16, 32, 32)
        self.b3 = Inception(256, 128, 128, 192, 32, 96, 64)

        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = Inception(480, 192,  96, 208, 16,  48,  64)
        self.b4 = Inception(512, 160, 112, 224, 24,  64,  64)
        self.c4 = Inception(512, 128, 128, 256, 24,  64,  64)
        self.d4 = Inception(512, 112, 144, 288, 32,  64,  64)
        self.e4 = Inception(528, 256, 160, 320, 32, 128, 128)

        self.a5 = Inception(832, 256, 160, 320, 32, 128, 128)
        self.b5 = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(1024, 10)

    def forward(self, x):
        out = self.pre_layers(x)
        out = self.a3(out)
        out = self.b3(out)
        out = self.maxpool(out)
        out = self.a4(out)
        out = self.b4(out)
        out = self.c4(out)
        out = self.d4(out)
        out = self.e4(out)
        out = self.maxpool(out)
        out = self.a5(out)
        out = self.b5(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


'''def test():
    net = GoogLeNet()
    x = torch.randn(1,3,32,32)
    y = net(x)
    print(y.size())
    print(y)

test()'''

'def test():\n    net = GoogLeNet()\n    x = torch.randn(1,3,32,32)\n    y = net(x)\n    print(y.size())\n    print(y)\n\ntest()'

# Preparing DATASET

In [0]:
import torchvision
import torchvision.transforms as transforms
'''
transforms.Normalize does the following for each channel:
image = (image - mean) / std
Normalization does helps CNN perform better.
Normalization helps get data within a range and reduces the skewness which helps learn faster and better
'''
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


==> Preparing data..


In [0]:
import torch.optim as optim
'''Parameters of DataLoader
        dataset (Dataset): dataset from which to load the data.
        batch_size (int, optional): how many samples per batch to load
            (default: ``1``).
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: ``False``).
        num_workers (int, optional): how many subprocesses to use for data
            loading. ``0`` means that the data will be loaded in the main process.
            (default: ``0``)
'''

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


# Training GoogleNet ON CIFAR-10

In [0]:
import torch.backends.cudnn as cudnn
#from utils import progress_bar


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
print('==> Building model..')
net = GoogLeNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
#Loss and Optimization
criterion = nn.CrossEntropyLoss() # It is useful when training a classification problem with `C` classes.
optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train() # Sets the model in training mode
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        #print(inputs.size())
        #print(targets)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() # UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
#Testing 
def test(epoch):
    global best_acc
    net.eval() # Sets the model in evaluation mode
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))


==> Building model..


In [0]:
for epoch in range(start_epoch, start_epoch+2):
    train(epoch)
    test(epoch)


Epoch: 0
0 391 Loss: 2.309 | Acc: 10.938% (14/128)
1 391 Loss: 2.313 | Acc: 10.938% (28/256)
2 391 Loss: 2.295 | Acc: 12.240% (47/384)
3 391 Loss: 2.285 | Acc: 12.305% (63/512)
4 391 Loss: 2.254 | Acc: 13.281% (85/640)
5 391 Loss: 2.237 | Acc: 13.802% (106/768)
6 391 Loss: 2.207 | Acc: 15.290% (137/896)
7 391 Loss: 2.172 | Acc: 17.188% (176/1024)
8 391 Loss: 2.151 | Acc: 18.056% (208/1152)
9 391 Loss: 2.134 | Acc: 18.984% (243/1280)
10 391 Loss: 2.149 | Acc: 18.963% (267/1408)
11 391 Loss: 2.147 | Acc: 19.466% (299/1536)
12 391 Loss: 2.148 | Acc: 19.832% (330/1664)
13 391 Loss: 2.167 | Acc: 19.754% (354/1792)
14 391 Loss: 2.170 | Acc: 19.896% (382/1920)
15 391 Loss: 2.164 | Acc: 20.264% (415/2048)
16 391 Loss: 2.154 | Acc: 20.634% (449/2176)
17 391 Loss: 2.144 | Acc: 21.224% (489/2304)
18 391 Loss: 2.145 | Acc: 21.423% (521/2432)
19 391 Loss: 2.140 | Acc: 21.641% (554/2560)
20 391 Loss: 2.134 | Acc: 21.912% (589/2688)
21 391 Loss: 2.122 | Acc: 22.301% (628/2816)
22 391 Loss: 2.123 | A

# GoogleNet ( Inception V2 ) 

1.   Replaced one 5x5 Kernal with 2 (3x3)



In [0]:
'''GoogLeNet with PyTorch.'''
import torch
import torch.nn as nn
import torch.nn.functional as F
class InceptionV2(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(InceptionV2, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )
        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )
        #print(n5x5)
        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )
        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
        
    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        '''print(y1.size())
        print(y2.size())
        print(y3.size())
        print(y4.size())'''
        return torch.cat([y1,y2,y3,y4], 1)
class GoogLeNet2(nn.Module):
    def __init__(self):
        super(GoogLeNet2, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )
        self.a3 = InceptionV2(192,  64,  96, 128, 16, 32, 32)
        self.b3 = InceptionV2(256, 128, 128, 192, 32, 96, 64)

        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = InceptionV2(480, 192,  96, 208, 16,  48,  64)
        self.b4 = InceptionV2(512, 160, 112, 224, 24,  64,  64)
        self.c4 = InceptionV2(512, 128, 128, 256, 24,  64,  64)
        self.d4 = InceptionV2(512, 112, 144, 288, 32,  64,  64)
        self.e4 = InceptionV2(528, 256, 160, 320, 32, 128, 128)

        self.a5 = InceptionV2(832, 256, 160, 320, 32, 128, 128)
        self.b5 = InceptionV2(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(1024, 10)

    def forward(self, x):
        out = self.pre_layers(x)
        out = self.a3(out)
        out = self.b3(out)
        out = self.maxpool(out)
        out = self.a4(out)
        out = self.b4(out)
        out = self.c4(out)
        out = self.d4(out)
        out = self.e4(out)
        out = self.maxpool(out)
        out = self.a5(out)
        out = self.b5(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
'''def test():
    net = GoogLeNet2()
    x = torch.randn(1,3,32,32)
    y = net(x)
    print(y.size())
    print(y)

test()'''

torch.Size([1, 10])
tensor([[-0.1829, -0.3134, -0.3924, -0.1104, -0.1087, -0.1274,  0.1873, -0.0997,
         -0.2290,  0.2572]], grad_fn=<AddmmBackward>)


# Training GoogLeNet2

In [0]:
import torch.backends.cudnn as cudnn
#from utils import progress_bar


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
print('==> Building model..')
net2 = GoogLeNet2()
net2 = net2.to(device)
if device == 'cuda':
    net2 = torch.nn.DataParallel(net2)
    cudnn.benchmark = True
#Loss and Optimization
criterion = nn.CrossEntropyLoss() # It is useful when training a classification problem with `C` classes.
optimizer = optim.SGD(net2.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)

# Training
def train2(epoch):
    print('\nEpoch: %d' % epoch)
    net2.train() # Sets the model in training mode
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        #print(inputs.size())
        #print(targets)
        optimizer.zero_grad()
        outputs = net2(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() # UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
#Testing 
def test2(epoch):
    global best_acc
    net2.eval() # Sets the model in evaluation mode
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net2(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))


==> Building model..


In [0]:
for epoch in range(start_epoch, start_epoch+2):
    train2(epoch)
    test2(epoch)


Epoch: 0
0 391 Loss: 2.321 | Acc: 7.031% (9/128)
1 391 Loss: 2.297 | Acc: 12.891% (33/256)
2 391 Loss: 2.295 | Acc: 15.365% (59/384)
3 391 Loss: 2.250 | Acc: 16.602% (85/512)
4 391 Loss: 2.226 | Acc: 18.594% (119/640)
5 391 Loss: 2.218 | Acc: 19.141% (147/768)
6 391 Loss: 2.170 | Acc: 20.759% (186/896)
7 391 Loss: 2.170 | Acc: 20.215% (207/1024)
8 391 Loss: 2.171 | Acc: 20.833% (240/1152)
9 391 Loss: 2.176 | Acc: 20.859% (267/1280)
10 391 Loss: 2.170 | Acc: 21.094% (297/1408)
11 391 Loss: 2.151 | Acc: 21.484% (330/1536)
12 391 Loss: 2.145 | Acc: 21.875% (364/1664)
13 391 Loss: 2.129 | Acc: 22.210% (398/1792)
14 391 Loss: 2.120 | Acc: 22.760% (437/1920)
15 391 Loss: 2.100 | Acc: 23.389% (479/2048)
16 391 Loss: 2.085 | Acc: 24.219% (527/2176)
17 391 Loss: 2.077 | Acc: 24.653% (568/2304)
18 391 Loss: 2.067 | Acc: 25.164% (612/2432)
19 391 Loss: 2.059 | Acc: 25.469% (652/2560)
20 391 Loss: 2.053 | Acc: 25.893% (696/2688)
21 391 Loss: 2.043 | Acc: 26.207% (738/2816)
22 391 Loss: 2.045 | Ac